In [1]:
import pandas as pd
import numpy as np

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as pl

matplotlib.rc('text', usetex=True)
pl.rcParams['text.latex.preamble'] = [
    r'\usepackage{tgheros}',    
    r'\usepackage[eulergreek]{sansmath}',   
    r'\sansmath',
    r'\usepackage{siunitx}',    
    r'\sisetup{detect-all}'
]

# library to generate the map
import folium
import folium.plugins as plugins

# generates the legend 
from utils.legend import template_macro

# custom date-time util
from utils.date_time import yyyy_mm_dd

import itertools as it

In [2]:
locs = pd.read_csv('data/static_locs.csv', sep=';', 
                   dtype={'latitude': np.float64, 'longitude': np.float64})
sens = pd.read_csv('data/sensor_locs.csv', sep=';', 
                   dtype={'latitude': np.float64, 'longitude': np.float64})

nox_all = pd.read_csv('data/NOX_Juli-Sept.csv', sep=',', decimal=",")

In [3]:
days, weekday = ['180704', '180708'], ['Mittwoch','Sonntag']
time_pts = [7,13,16,19,24,29,34,39,44]

for wd,day in zip(weekday,days):
    '''main loop; generate maps for every time point on every day'''
    
    # load traffic data    
    df = pd.read_csv('data/30m/FFM_DEZ_'+day+'_30min.csv', 
                     error_bad_lines=False, encoding='ISO-8859-1', 
                     sep=',')
    
    # filter nox values for day in question
    # convert 180703 to 2018-07-03
    noxd = nox_all[(nox_all['time'] > yyyy_mm_dd(day)+' 00:00') \
                   & (nox_all['time'] < yyyy_mm_dd(day)+' 23:59')]
    
    
    for tp in time_pts:
        
        # -------------- create nox curve graphic ---------------
        
        fname = day+'_%.2d'%(tp)
        
        fig, ax = pl.subplots()
        fig.set_size_inches(2,1)
        ax.plot(list(noxd['NOX']), color='#E37222', lw=2);
        
        #print(list(noxd['time'])[tp])
        pl.xticks([7,23,39], labels=['4:00','12:00','20:00'])

        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')

        ax.annotate('',xy=(tp,list(noxd['NOX'])[tp]), 
                    xytext=(tp,list(noxd['NOX'])[tp]+40),
                    arrowprops=dict(facecolor='black', 
                                    shrink=0.2, headwidth=3, 
                                    headlength=5, width=1))

        ax.set_ylim(0,130)
            
        fig.savefig('maps/original_data/gfx/'+fname+'.png', dpi=200, bbox_inches='tight')
        
        
        # ---------------- create folium map --------------------        
        
        folium_map = folium.Map(location=[locs['latitude'][0], locs['longitude'][0]],
                                zoom_start=14, 
                                #tiles="CartoDB dark_matter")
                                tiles='cartodbpositron')
        
        fail = 0

        for index, row in locs.iterrows():

            try:
                if int(df.iloc[tp][str(int(row['ElemUID']))]) > 0:

                    popup_text = """<br>
                                Spulen-ID: {}<br>
                                Fahrten: {}"""
                    popup_text = popup_text.format(int(row['ElemUID']),
                                                   int(df.iloc[tp][str(int(row['ElemUID']))]))

                    marker = folium.CircleMarker(location=[row['latitude'], row['longitude']], 
                                                 radius=int(df.iloc[tp][str(int(row['ElemUID']))])/10.,
                                                 popup=popup_text, fill=True)
                    marker.add_to(folium_map)

            except KeyError:
                fail+=1


        print('Could not add ', fail, 'ElemUIDs')  

        for index, row in sens.iterrows():
            if index==1:
                marker = folium.CircleMarker(location=[row['latitude'], row['longitude']], 
                                             color="#E37222", fill=True, 
                                             radius=noxd.iloc[tp]['NOX']/5)
                marker.add_to(folium_map) 


        print(noxd.iloc[tp]['NOX'])
        
        macro = template_macro(wd+' '+day[5]+'.'+day[3]+'.'+'2018')           
        folium_map.get_root().add_child(macro)

        url = '../gfx/'+fname+'.png'

        plugins.FloatImage(url, bottom=2.5, left=2.5).add_to(folium_map)

        #folium.TileLayer('stamentoner').add_to(folium_map)

        folium_map.save('maps/original_data/'+fname+'.html')

        folium_map


Could not add  117 ElemUIDs
37.6
Could not add  117 ElemUIDs
87.6
Could not add  117 ElemUIDs
94.3
Could not add  117 ElemUIDs
89.0
Could not add  117 ElemUIDs
53.1
Could not add  117 ElemUIDs
43.7
Could not add  117 ElemUIDs
63.7
Could not add  117 ElemUIDs
32.3
Could not add  117 ElemUIDs
59.3
Could not add  117 ElemUIDs
14.8
Could not add  117 ElemUIDs
12.5
Could not add  117 ElemUIDs
17.9
Could not add  117 ElemUIDs
13.0
Could not add  117 ElemUIDs
11.2
Could not add  117 ElemUIDs
16.6
Could not add  117 ElemUIDs
27.8
Could not add  117 ElemUIDs
47.5
Could not add  117 ElemUIDs
29.5


In [4]:
# show one example
folium_map